UI to work with SBI stuff

In [1]:
import os
import subprocess

def source_lmod_script(script_path):
    """
    Source an Lmod/module script and import environment variables into Python safely,
    suppressing terminal warnings.
    """
    # Use a non-interactive login shell (bash -l), redirect errors
    command = f'bash -l -c "source {script_path} >/dev/null 2>&1; printenv -0"'
    
    proc = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    out, _ = proc.communicate()
    
    # Parse null-separated environment variables
    for env_var in out.split(b'\0'):
        if env_var:
            key, _, value = env_var.partition(b'=')
            os.environ[key.decode()] = value.decode()

# Example usage
M3_BUILD_DIR = "/home/henryi/scratch/venvs/.venv_sbi/bin/"
TUTORIAL_BUILD_DIR = M3_BUILD_DIR
source_lmod_script(f"{M3_BUILD_DIR}/setup.MaCh3.sh")
source_lmod_script(f"{TUTORIAL_BUILD_DIR}/setup.MaCh3Tutorial.sh")
os.environ["OMP_NUM_THREADS"] = "8"


In [2]:
from mach3sbitools.ui.sbi_ui import MaCh3SbiUI
from mach3sbitools.utils.device_handler import TorchDeviceHander

from pathlib import Path

from sbi.utils import BoxUniform

In [3]:
input_file = Path("/home/henryi/sft/MaCh3Tutorial/TutorialConfigs/FitterConfig.yaml")
ui = MaCh3SbiUI(str(input_file))
device_handler = TorchDeviceHander()

[Monitor.cpp][info] ##################################
[Monitor.cpp][info] Welcome to:  
[Monitor.cpp][info]   __  __        _____ _     ____  
[Monitor.cpp][info]  |  \/  |      / ____| |   |___ \ 
[Monitor.cpp][info]  | \  / | __ _| |    | |__   __) |
[Monitor.cpp][info]  | |\/| |/ _` | |    | '_ \ |__ < 
[Monitor.cpp][info]  | |  | | (_| | |____| | | |___) |
[Monitor.cpp][info]  |_|  |_|\__,_|\_____|_| |_|____/ 
[Monitor.cpp][info] Version: 2.3.1
[Monitor.cpp][info] ##################################
[Monitor.cpp][info] Using following CPU:
[Monitor.cpp][info] model name	: AMD EPYC 9454 48-Core Processor
[Monitor.cpp][info] cpu MHz		: 2750.000
[Monitor.cpp][info] Architecture:                         x86_64
[Monitor.cpp][info] L1d cache:                            1.5 MiB (48 instances)
[Monitor.cpp][info] L1i cache:                            1.5 MiB (48 instances)
[Monitor.cpp][info] L2 cache:                             48 MiB (48 instances)
[Monitor.cpp][info] L3 cache:         

cat: write error: Broken pipe
cat: write error: Broken pipe


In [4]:
# Set up the prior 
lower_bounds = device_handler.to_tensor([0,   0,   0,    0, 0, 0, 0, 0, -1, -1])
upper_bounds = device_handler.to_tensor([5, 5, 5, 2, 2, 2, 3, 2, 1,   1])

prior = BoxUniform(lower_bounds, upper_bounds)
ui.set_prior(prior)

In [9]:
# Fit settings!
FIT_TYPE='AutomaticTransform'
N_ROUNDS = 50

training_settings = {}

ttree_settings = {}

FIT_ARGS = {
    "TrainingSettings": training_settings,    
}

In [ ]:
# Now we can run the fit!
ui.run_fit(FIT_TYPE, N_ROUNDS, 5000, {}, {})

 Neural network successfully converged after 359 epochs.

  0%|          | 0/5000 [00:00<?, ?it/s]

 Neural network successfully converged after 162 epochs.

  0%|          | 0/5000 [00:00<?, ?it/s]

 Neural network successfully converged after 70 epochs.

  0%|          | 0/5000 [00:00<?, ?it/s]

 Neural network successfully converged after 37 epochs.

  0%|          | 0/5000 [00:00<?, ?it/s]

 Training neural network. Epochs trained: 59

In [ ]:


s = ui.fitter.posterior.sample((10000,), device_handler.to_tensor(ui.fitter.x0))

from matplotlib import pyplot as plt


In [ ]:
plt.hist(s[:,0].cpu())